In [422]:
from sqlanalyzer import column_parser
import re
import json
import pandas as pd

### debug

In [433]:
query = """SELECT
      b.*
      FROM
      (
        SELECT
        MAX(dt) AS dt
        FROM
        analytics.service_endpoint_mapping
      ) a
      LEFT JOIN sfdc.accounts sfdc_accounts ON sfdc_accounts.id = b.id
      INNER JOIN
      analytics.service_endpoint_mapping b
      ON
      a.dt = b.dt
      AND a.id = b.id
      LEFT JOIN (
        SELECT
        account,
        CONCAT_WS(',', COLLECT_SET(LOWER(service))) AS service
        FROM
        web_mobile_studio
        GROUP BY
        account
      ) c
      ON
      a.account = c.account AND c.service = 'studio'
      WHERE a.id = '123'
      AND b.id = '234'
      GROUP BY 1,2,3
      ORDER BY 3,4
"""


In [375]:
query = """SELECT a.*,
             b.*,
             c.*,
             d.*
      FROM
        (SELECT DISTINCT anonymous_id,
                         user_id
         FROM mapbox_customer_data.segment_identifies
         WHERE dt >= '2018-07-01'
           AND anonymous_id IS NOT NULL
           AND user_id IS NOT NULL ) a
      LEFT JOIN
        (SELECT id,
                email,
                created
         FROM mapbox_customer_data.accounts
         WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON a.user_id = b.id
      LEFT JOIN
        (SELECT anonymous_id AS anon_id_ad,
                context_campaign_name,
                min(TIMESTAMP) AS min_exposure
         FROM mapbox_customer_data.segment_pages
         WHERE dt >= '2018-07-01'
           AND context_campaign_name IS NOT NULL
         GROUP BY 1,
                  2) c ON a.anonymous_id = c.anon_id_ad
      LEFT JOIN
        (SELECT DISTINCT anonymous_id AS anon_id_event,
                         original_timestamp,
                         event,
                         context_traits_email
         FROM mapbox_customer_data.segment_tracks
         WHERE dt >= '2018-07-01'
           AND event LIKE 'submitted_%form'
           AND context_traits_email IS NOT NULL ) d ON a.anonymous_id = d.anon_id_event
"""

In [376]:
print(query)

SELECT a.*,
             b.*,
             c.*,
             d.*
      FROM
        (SELECT DISTINCT anonymous_id,
                         user_id
         FROM mapbox_customer_data.segment_identifies
         WHERE dt >= '2018-07-01'
           AND anonymous_id IS NOT NULL
           AND user_id IS NOT NULL ) a
      LEFT JOIN
        (SELECT id,
                email,
                created
         FROM mapbox_customer_data.accounts
         WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON a.user_id = b.id
      LEFT JOIN
        (SELECT anonymous_id AS anon_id_ad,
                context_campaign_name,
                min(TIMESTAMP) AS min_exposure
         FROM mapbox_customer_data.segment_pages
         WHERE dt >= '2018-07-01'
           AND context_campaign_name IS NOT NULL
         GROUP BY 1,
                  2) c ON a.anonymous_id = c.anon_id_ad
      LEFT JOIN
        (SELECT DISTINCT anonymous_id AS anon_id_event,
                         original_timesta

In [423]:
def within(num, rng):
    if num >= min(rng) and num <= max(rng) and min(rng) < max(rng): return 1
    else: return 0


def is_cte(query):
    return query.startswith('WITH')


def clean_dict(query_dict):

    for k,v in query_dict.items(): 
        if isinstance(v, dict) and len(v.keys()) == 1 and 'main' in v.keys():
            query_dict[k] = v['main']
            
    return query_dict


In [424]:
def main_query(query_list, sub_query_pos):

    l = []
    for i in range(len(query_list)): 
        count = 0
        for pair in sorted(sub_query_pos):
            count += within(i, pair)
        if count == 0:
            l.append(i)

    return l


def has_child(formatted_query):

    count = 0
    deleveled_list = delevel(formatted_query.split('\n'))
    if len(deleveled_list.keys()) > 1:
        for _,v in deleveled_list.items():
            if v != {}: count += 1

    return count != 0


In [425]:
def delevel(query_list):

    sub_query = {}
    pos_join, pos_where = _get_joins_pos(query_list)
    alias_pos = _get_alias_pos(query_list, pos_join, pos_where)
    sub_query_pos = list(zip(pos_join, alias_pos))
    sub_query, keep = _parse_sub_query(query_list, sub_query_pos)
    main_query_pos = main_query(query_list, sub_query_pos)
    if main_query_pos != []:
        sub_query['main'] = '\n'.join([query_list[p] for p in main_query_pos])
    sub_query['main'] = sub_query['main'] + ' ' + '\n'.join(keep)

    return sub_query


## new: start get_sub_queries

In [426]:
query = """SELECT * 
, CASE WHEN service = 'directionsmatrix' THEN
   CASE WHEN (sources = 'all' or sources is null or trim(sources) = '') AND (destinations = 'all' or destinations is null or trim(destinations) = '') then query_cardin*query_cardin
        WHEN (sources = 'all' or sources is null  or trim(sources) = '') and destinations != 'all' and destinations is not null and trim(destinations) != '' then query_cardin*destinations_cardin
        WHEN sources != 'all' and sources is not null and trim(sources) != '' and (destinations = 'all' or destinations is null or trim(destinations) = '') then sources_cardin*query_cardin
        WHEN sources != 'all' and sources is not null and trim(sources) != '' and destinations != 'all' and destinations is not null and trim(destinations) != '' then sources_cardin*destinations_cardin
        ELSE NULL end 
   WHEN service in ('geocode', 'permanentgeocode') THEN query_cardin
        ELSE 1 END as total_subunits
FROM (
SELECT * 
, replace(regexp_extract(lower(cs_uri_query), '(sources=)(.+?)((?=&)|$)', 2), '%253b', ';') as sources
, cardinality(split(replace(regexp_extract(lower(cs_uri_query), '(sources=)(.+?)((?=&)|$)', 2), '%253b', ';'), ';')) as sources_cardin
, CASE WHEN cs_uri_query like '%destinations&%' then '1' ELSE
replace(regexp_extract(lower(cs_uri_query), '(destinations=)(.+?)((?=&)|$)', 2), '%253b', ';') end as destinations
, CASE WHEN cs_uri_query like '%destinations&%' then 1 ELSE
cardinality(split(replace(regexp_extract(lower(cs_uri_query), '(destinations=)(.+?)((?=&)|$)', 2), '%253b', ';'), ';')) end as destinations_cardin
--, split(replace(replace(lower(query), '%253b', ';'), '%252c', ','), ';') as query_mod
, cardinality(split(replace(replace(lower(query), '%253b', ';'), '%252c', ','), ';')) as query_cardin 
FROM {cloudfront_logs_china_dataset}
WHERE dt = '{run_date}'
AND cast(hr as BIGINT) in {hr_block} 
--AND service in ('geocode', 'directionsmatrix', 'permanentgeocode')
)
"""

In [428]:
def get_sub_query(query_list):
    pos_delete, pos_where = [len(query_list)-1], len(query_list)

    for i, line in enumerate(query_list):
        if line.startswith('ORDER') or line.startswith('GROUP'):
            pos_delete.append(i)    
        elif line.startswith('WHERE'):
            pos_where = i

    end_of_query = min(pos_delete) 
    
    copy_query_list = query_list.copy()
    main = next((s for s in copy_query_list if s.startswith('FROM')), 'end of query')
    main_pos = copy_query_list.index(main)
    main_query = copy_query_list[:main_pos+1]
    main_query.extend(copy_query_list[pos_where:end_of_query])
    del copy_query_list[:main_pos]
    del copy_query_list[(pos_where-main_pos):]
    
    return copy_query_list


In [429]:
def landmark(line):
    for syntax in ['FROM', 'LEFT JOIN', 'INNER JOIN', 'OUTER JOIN', 'RIGHT JOIN', 'CROSS JOIN', 'FULL JOIN', 'FULL OUTER JOIN']:
        if line.startswith(syntax):
            return True
        

In [430]:
def divide(copy_query_list):
    sub_join = []
    for i, line in enumerate(copy_query_list): 

        if landmark(line):
            sub_join.append(line)
            del copy_query_list[:i+1]
            join_query = next((s for s in copy_query_list if not s.startswith(' ')), 'end of query')
            try:
                join_pos = copy_query_list.index(join_query)
                sub_join.extend(copy_query_list[:join_pos])
                del copy_query_list[:join_pos]
                break
            except: 
                sub_join.extend(copy_query_list)
                del copy_query_list[:]
                break

            
    return sub_join, copy_query_list


In [431]:
def get_sub_dict(copy_query_list):
    i = 0
    join_dict = {}
    new_copy_query_list = copy_query_list
    sub_join = [1]
    while sub_join != []:
        i += 1
        sub_join, new_copy_query_list = divide(new_copy_query_list)
        if sub_join != []:
            join_dict['join_{}'.format(i)] = '\n'.join(sub_join)
            
    return join_dict


In [435]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list_0 = formatted_query.split('\n')

copy_query_list = get_sub_query(query_list_0)
join_dict = get_sub_dict(copy_query_list)
join_dict

{'join_1': 'FROM\n  (SELECT MAX(dt) AS dt\n   FROM analytics.service_endpoint_mapping) a',
 'join_2': 'LEFT JOIN sfdc.accounts sfdc_accounts ON sfdc_accounts.id = b.id',
 'join_3': 'INNER JOIN analytics.service_endpoint_mapping b ON a.dt = b.dt',
 'join_4': "LEFT JOIN\n  (SELECT account,\n          CONCAT_WS(',', COLLECT_SET(LOWER(service))) AS service\n   FROM web_mobile_studio\n   GROUP BY account) c ON a.account = c.account"}

## end get_sub_queries